<a href="https://colab.research.google.com/github/ParnianH98/Movie-Recommender-System/blob/main/Movie_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
df_cred = pd.read_csv(r'/gdrive/My Drive/AI HW/tmdb_5000_credits.csv')
df_movie = pd.read_csv(r'/gdrive/My Drive/AI HW/tmdb_5000_movies.csv')

In [ ]:
df_cred

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
df_movie.head(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avenge

In [ ]:
df_cred.columns = ['id','tittle','cast','crew']
df = df_movie.merge(df_cred,on ='id')

# first approach

In [ ]:
filtered = df['vote_count'].quantile(0.8)
filtered

957.6000000000004

In [ ]:
q_movies = df.copy().loc[df['vote_count'] >= filtered]
q_movies.shape

(961, 23)

In [ ]:
meanVote = df['vote_average'].mean()
meanVote

6.092171559442011

In [ ]:
def weighted_rating(data, f = filtered, mv = meanVote):
    vc = data['vote_count']
    vav = data['vote_average']
    # Calculation based on the IMDB formula
    return (vc/(vc + f) * vav) + (f/(f + vc) * mv)

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 10 movies based on the score value
q_movies[['title', 'genres' , 'overview' , 'vote_count', 'vote_average', 'score']].head(10)


,title,genres,overview,vote_count,vote_average,score
1881,The Shawshank Redemption,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",Framed in the 1940s for the double murder of h...,8205,8.5,8.248353
662,Fight Club,"[{""id"": 18, ""name"": ""Drama""}]",A ticking-time-bomb insomniac and a slippery s...,9413,8.3,8.096134
3337,The Godfather,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","Spanning the years 1945 to 1955, a chronicle o...",5893,8.4,8.077404
3232,Pulp Fiction,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...","A burger-loving hit man, his philosophical par...",8428,8.3,8.074738
65,The Dark Knight,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",Batman raises the stakes in his war on crime. ...,12002,8.2,8.044250
809,Forrest Gump,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",A man with a low IQ has accomplished great thi...,7927,8.2,7.972814
96,Inception,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...","Cobb, a skilled thief who commits corporate es...",13752,8.1,7.969290
95,Interstellar,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",Interstellar chronicles the adventures of a gr...,10867,8.1,7.937399
1990,The Empire Strikes Back,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...","The epic saga continues as Luke Skywalker, in ...",5879,8.2,7.904757
1818,Schindler's List,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",The true story of how businessman Oskar Schind...,4329,8.3,7.900080


# second approach

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfiDF = TfidfVectorizer(stop_words='english')

#Overview is my choice
#Anybody else can use other facors to find similarities such as 'genre'
#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfiDF_matrix = tfiDF.fit_transform(df['overview'])

#Output the shape of tfiDF_matrix
tfiDF_matrix.shape

(4803, 20978)

there are over 20000 words to descrie the 'overview' of 4803 films 

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix using linear_kernel which is similar to cosine_similarity but this is faster
cosine = linear_kernel(tfiDF_matrix, tfiDF_matrix)

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine=cosine):
    # Get the index of the movie that matches the title
    #step 1
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    #step 2
    sim_scores = list(enumerate(cosine[idx]))

    # Sort the movies based on the similarity scores
    #step 3
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    # step 4
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df[['title', 'genres' , 'overview' , 'vote_count', 'vote_average']].iloc[movie_indices]

In [ ]:
get_recommendations('Inception')

,title,genres,overview,vote_count,vote_average
2897,Cypher,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...","An unsuspecting, disenchanted man finds himsel...",192,6.7
134,Mission: Impossible - Rogue Nation,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Ethan and team take on their most impossible m...,3224,7.1
1930,Stone Cold,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","Joe Huff (Brian Bosworth) is a tough, loner co...",34,5.7
914,Central Intelligence,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",After he reunites with an old pal through Face...,1650,6.2
1683,Pitch Perfect 2,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10402, ""...","The Bellas are back, and they are better than ...",1940,6.8
1248,At First Sight,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",A blind man has an operation to regain his sig...,48,5.9
1512,A History of Violence,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",An average family is thrust into the spotlight...,832,6.9
2389,Renaissance,"[{""id"": 28, ""name"": ""Action""}, {""id"": 16, ""nam...",To find Ilona and unlock the secrets of her di...,77,6.7
1803,Blood and Wine,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",A man who has failed as a father and husband c...,34,5.5
1267,Duplex,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...","When a young couple buys their dream home, the...",482,5.9
